# **Data Cleaning Notebook**

## Objectives

- Identify and assess missing values in the dataset
- Handle missing data through imputation or removal
- Prepare a clean dataset for modeling and further analysis

## Inputs

- outputs/datasets/collection/HousePrices.csv

## Outputs

- Cleaned full dataset: outputs/datasets/cleaned/HousePricesCleaned.csv
- Cleaned train/test splits: outputs/datasets/cleaned/TrainSetCleaned.csv, outputs/datasets/cleaned/TestSetCleaned.csv
- Data cleaning pipeline: outputs/ml_pipeline/data_cleaning/dataCleaning_pipeline.pkl


---

# Change working directory

In [ ]:
import os

current_dir = os.getcwd()
os.chdir(os.path.dirname(current_dir))  # set project root
print("Current directory:", os.getcwd())

---

## Load Collected Data

In [ ]:
import pandas as pd

df = pd.read_csv("outputs/datasets/collection/HousePrices.csv")
df.head(5)

---

## Data Exploration

In Data Cleaning you are interested to check the distribution and shape of a variable with missing data.

In [ ]:
vars_with_missing_data = df.columns[df.isna().sum() > 0].to_list()
vars_with_missing_data

In [ ]:
from ydata_profiling import ProfileReport

if vars_with_missing_data:
    profile = ProfileReport(df=df[vars_with_missing_data], minimal=True)
    profile.to_notebook_iframe()
else:
    print("There are no variables with missing data")

---

## Correlation and PPS Analysis

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps

warnings.filterwarnings("ignore")

def heatmap_corr(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=bool)
        mask[np.triu_indices_from(mask)] = True
        mask[abs(df) < threshold] = True

        fig, axes = plt.subplots(figsize=figsize)
        sns.heatmap(
            df,
            annot=True,
            xticklabels=True,
            yticklabels=True,
            mask=mask,
            cmap="viridis",
            annot_kws={"size": font_annot},
            ax=axes,
            linewidth=0.5,
        )
        axes.set_yticklabels(df.columns, rotation=0)
        plt.ylim(len(df.columns), 0)
        plt.show()


def heatmap_pps(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=bool)
        mask[abs(df) < threshold] = True
        fig, ax = plt.subplots(figsize=figsize)
        ax = sns.heatmap(
            df,
            annot=True,
            xticklabels=True,
            yticklabels=True,
            mask=mask,
            cmap="rocket_r",
            annot_kws={"size": font_annot},
            linewidth=0.05,
            linecolor="grey",
        )
        plt.ylim(len(df.columns), 0)
        plt.show()


def CalculateCorrAndPPS(df):
    df_corr_spearman = df.corr(method="spearman", numeric_only=True)
    df_corr_pearson = df.corr(method="pearson", numeric_only=True)

    pps_matrix_raw = pps.matrix(df)
    pps_matrix = pps_matrix_raw.filter(["x", "y", "ppscore"]).pivot(
        columns="x", index="y", values="ppscore"
    )

    pps_score_stats = (
        pps_matrix_raw.query("ppscore < 1").filter(["ppscore"]).describe().T
    )
    print("PPS threshold - check PPS score IQR to decide threshold for heatmap \n")
    print(pps_score_stats.round(3))

    return df_corr_pearson, df_corr_spearman, pps_matrix


def DisplayCorrAndPPS(
    df_corr_pearson,
    df_corr_spearman,
    pps_matrix,
    CorrThreshold,
    PPS_Threshold,
    figsize=(20, 12),
    font_annot=8,
):

    print("\n")
    print(
        "* Analyse how the target variable for your ML models are correlated with other variables (features and target)"
    )
    print(
        "* Analyse multi-colinearity, that is, how the features are correlated among themselves"
    )

    print("\n")
    print("*** Heatmap: Spearman Correlation ***")
    print("It evaluates monotonic relationship \n")
    heatmap_corr(
        df=df_corr_spearman,
        threshold=CorrThreshold,
        figsize=figsize,
        font_annot=font_annot,
    )

    print("\n")
    print("*** Heatmap: Pearson Correlation ***")
    print("It evaluates the linear relationship between two continuous variables \n")
    heatmap_corr(
        df=df_corr_pearson,
        threshold=CorrThreshold,
        figsize=figsize,
        font_annot=font_annot,
    )

    print("\n")
    print("*** Heatmap: Power Predictive Score (PPS) ***")
    print(
        f"PPS detects linear or non-linear relationships between two columns.\n"
        f"The score ranges from 0 (no predictive power) to 1 (perfect predictive power) \n"
    )
    heatmap_pps(
        df=pps_matrix, threshold=PPS_Threshold, figsize=figsize, font_annot=font_annot
    )

Calculate Correlations and Power Predictive Score.

In [ ]:
df_corr_pearson, df_corr_spearman, pps_matrix = CalculateCorrAndPPS(df)

Display at Heatmaps

In [ ]:
DisplayCorrAndPPS(
    df_corr_pearson=df_corr_pearson,
    df_corr_spearman=df_corr_spearman,
    pps_matrix=pps_matrix,
    CorrThreshold=0.4,
    PPS_Threshold=0.2,
    figsize=(12, 10),
    font_annot=10,
)

---

## Data Cleaning

Assessing Missing Data Levels

- Custom function to display missing data levels in a DataFrame, it shows the absolute levels, relative levels and data type.

In [ ]:
def EvaluateMissingData(df):
    """
    Function to evaluate data with missing values, including most frequent value (mode).
    """
    missing_data_absolute = df.isnull().sum()
    missing_data_percentage = round(missing_data_absolute / len(df) * 100, 2)

    most_frequent_values = df.apply(
        lambda col: (
            col.mode(dropna=True)[0] if not col.mode(dropna=True).empty else "No mode"
        )
    )

    df_missing_data = (
        pd.DataFrame(
            data={
                "RowsWithMissingData": missing_data_absolute,
                "PercentageOfDataset": missing_data_percentage,
                "DataType": df.dtypes,
                "MostFrequentValue": most_frequent_values,
            }
        )
        .sort_values(by=["PercentageOfDataset"], ascending=False)
        .query("PercentageOfDataset > 0")
    )

    return df_missing_data

In [ ]:
EvaluateMissingData(df)

### Missing Data Evaluation
Our goal is to keep as much useful data as possible, while dealing with missing values properly. We looked at each column with missing values and decided whether to drop it or fill it in.

**Columns to Drop**
These columns are missing too many values, and they don't seem very helpful for predicting the house price. So we will drop them.

| Column          | Missing % | Reason                                  |
|-----------------|-----------|------------------------------------------|
| EnclosedPorch   | 90.68%    | Too many missing values, not very useful |
| WoodDeckSF      | 89.38%    | Also many missing values, not so helpful |


**Columns to Impute**
These columns are more useful, and the missing values can be filled in with simple methods like the most common value or the average/median.
Findings:

| Column          | Missing % | Fill With        | Reason |
|-----------------|-----------|------------------|--------|
| LotFrontage     | 17.74%    | Median           | Good for numbers like this |
| GarageFinish    | 16.10%    | 'unf' (unfinished) | Most common value |
| BsmtFinType1    | 9.93%     | 'unf'            | Most common value |
| BedroomAbvGr    | 6.78%     | Median           | Safe and simple |
| 2ndFlrSF        | 5.89%     | 0                | Most houses have no 2nd floor |
| GarageYrBlt     | 5.55%     | Median           | Close to house build year |
| BsmtExposure    | 2.60%     | 'no'             | Most common value |
| MasVnrArea      | 0.55%     | 0                | Most houses have 0 here |



## Drop Unnecessary Variables

In order to assess the effect of dropped data for each of the imputations we are about to make, we use CI's custom code:

In [ ]:
def DataCleaningEffect(df_original, df_cleaned, variables_applied_with_method):

    flag_count = 1  # Indicate plot number

    # distinguish between numerical and categorical variables
    categorical_variables = df_original.select_dtypes(exclude=["number"]).columns

    # scan over variables,
    # first on variables that you applied the method
    # if the variable is a numerical plot, a histogram if categorical plot a barplot
    for set_of_variables in [variables_applied_with_method]:
        print(
            "\n====================================================================================="
        )
        print(
            f"* Distribution Effect Analysis After Data Cleaning Method in the following variables:"
        )
        print(f"{set_of_variables} \n\n")

        for var in set_of_variables:
            if var in categorical_variables:  # it is categorical variable: barplot

                df1 = pd.DataFrame({"Type": "Original", "Value": df_original[var]})
                df2 = pd.DataFrame({"Type": "Cleaned", "Value": df_cleaned[var]})
                dfAux = pd.concat([df1, df2], axis=0)
                fig, axes = plt.subplots(figsize=(15, 5))
                sns.countplot(
                    hue="Type", data=dfAux, x="Value", palette=["#432371", "#FAAE7B"]
                )
                axes.set(title=f"Distribution Plot {flag_count}: {var}")
                plt.xticks(rotation=90)
                plt.legend()

            else:  # it is numerical variable: histogram

                fig, axes = plt.subplots(figsize=(10, 5))
                sns.histplot(
                    data=df_original,
                    x=var,
                    color="#432371",
                    label="Original",
                    kde=True,
                    element="step",
                    ax=axes,
                )
                sns.histplot(
                    data=df_cleaned,
                    x=var,
                    color="#FAAE7B",
                    label="Cleaned",
                    kde=True,
                    element="step",
                    ax=axes,
                )
                axes.set(title=f"Distribution Plot {flag_count}: {var}")
                plt.legend()

            plt.show()
            flag_count += 1

In [ ]:
from feature_engine.selection import DropFeatures

variables_method = ["EnclosedPorch", "WoodDeckSF"]
variables_method

imputer = DropFeatures(features_to_drop=variables_method)
df_method = imputer.fit_transform(df)
df_method.head(5)

## Impute Variables (handle missing data)

### Arbitrary Number Imputation

- The number '0' shall be imputed for 2ndFlrSF and MasVnrArea

In [ ]:
from feature_engine.imputation import ArbitraryNumberImputer

variables_method = ["2ndFlrSF", "MasVnrArea"]
variables_method

imputer = ArbitraryNumberImputer(arbitrary_number=0, variables=variables_method)
df_method = imputer.fit_transform(df)

DataCleaningEffect(
    df_original=df, df_cleaned=df_method, variables_applied_with_method=variables_method
)

We can see that the cleaned data has a similar distribution to the original data, indicating no major distortion by the cleaning.

### Median Imputation

The median shall be imputed for LotFrontage, BedroomAbvGr and GarageYrBlt.

In [ ]:
from feature_engine.imputation import MeanMedianImputer

variables_method = ["LotFrontage","BedroomAbvGr","GarageYrBlt"]
variables_method

imputer = MeanMedianImputer(imputation_method="median", variables=variables_method)
df_method = imputer.fit_transform(df)

DataCleaningEffect(
    df_original=df, df_cleaned=df_method, variables_applied_with_method=variables_method
)

While the distributions of the cleaned data are not identical to the original, they remain fairly consistent overall. Since the visual comparison shows no significant distortion, we consider the cleaning process appropriate.

### Categorical Imputation

- The value 'Unf', due to being the most common one, shall be imputed for BsmtFinType1 and GarageFinish.

In [ ]:
from feature_engine.imputation import CategoricalImputer

variables_method = ["BsmtFinType1", "GarageFinish"]
variables_method

imputer = CategoricalImputer(
    imputation_method="missing", fill_value="Unf", variables=variables_method
)
df_method = imputer.fit_transform(df)

DataCleaningEffect(
    df_original=df, df_cleaned=df_method, variables_applied_with_method=variables_method
)

As expected, the cleaned data shows more values of 'Unf', which is what the imputation was supposed to achieve.

The value 'No', due to being the most common one, shall be imputed for BsmtExposure.

In [ ]:
from feature_engine.imputation import CategoricalImputer

variables_method = ["BsmtExposure"]
variables_method

imputer = CategoricalImputer(
    imputation_method="missing", fill_value="No", variables=variables_method
)
df_method = imputer.fit_transform(df)

DataCleaningEffect(
    df_original=df, df_cleaned=df_method, variables_applied_with_method=variables_method
)

## Split Train and Test Set

In [ ]:
from sklearn.model_selection import train_test_split

# Split the final cleaned df_method into Train/Test
TrainSet, TestSet, _, __ = train_test_split(
    df_method, df_method["SalePrice"], test_size=0.2, random_state=0
)

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

In [ ]:
X_train.isnull().sum().sort_values(ascending=False).head()

- These cleaned datasets can now be used for Feature Engineering or Modeling

---

## Data Cleaning Pipeline

Apply the pipeline to the whole dataset to get cleaned data.

In [ ]:
from sklearn.pipeline import Pipeline
from feature_engine.imputation import (
    MeanMedianImputer,
    ArbitraryNumberImputer,
    CategoricalImputer,
)

cleaning_pipeline = Pipeline(
    steps=[
        (
            "median_imputer",
            MeanMedianImputer(
                imputation_method="median",
                variables=["LotFrontage", "GarageYrBlt", "BedroomAbvGr"],
            ),
        ),
        (
            "arbitrary_imputer",
            ArbitraryNumberImputer(
                arbitrary_number=0, variables=["2ndFlrSF", "MasVnrArea"]
            ),
        ),
        (
            "cat_imputer_unf",
            CategoricalImputer(
                imputation_method="missing",
                fill_value="Unf",
                variables=["GarageFinish", "BsmtFinType1"],
            ),
        ),
        (
            "cat_imputer_no",
            CategoricalImputer(
                imputation_method="missing", fill_value="No", variables=["BsmtExposure"]
            ),
        ),
    ]
)

## Apply the Pipeline to Train and Test Sets

In [ ]:
# Fit the pipeline only on training data
cleaning_pipeline.fit(X_train)

# Transform both train and test sets
X_train_cleaned = cleaning_pipeline.transform(X_train)
X_test_cleaned = cleaning_pipeline.transform(X_test)

# Optional: combine with target
TrainSetCleaned = X_train_cleaned.copy()
TrainSetCleaned["SalePrice"] = y_train.values

TestSetCleaned = X_test_cleaned.copy()
TestSetCleaned["SalePrice"] = y_test.values

FullCleaned = pd.concat([TrainSetCleaned, TestSetCleaned], axis=0)

---

## Save Cleaned Data and Pipeline

In [ ]:
import os
import joblib

# Create output folders
os.makedirs("outputs/datasets/cleaned", exist_ok=True)
os.makedirs("outputs/ml_pipeline/data_cleaning", exist_ok=True)

# Save cleaned data
TrainSetCleaned.to_csv("outputs/datasets/cleaned/TrainSetCleaned.csv", index=False)
TestSetCleaned.to_csv("outputs/datasets/cleaned/TestSetCleaned.csv", index=False)
FullCleaned.to_csv("outputs/datasets/cleaned/HousePricesCleaned.csv", index=False)

# Save pipeline
joblib.dump(
    cleaning_pipeline, "outputs/ml_pipeline/data_cleaning/dataCleaning_pipeline.pkl"
)

---

## Summary and Next Steps

**Summary**

- Assessed and handled missing values:
    - Median imputation: variables = ['LotFrontage', 'GarageYrBlt', 'BedroomAbvGr']
    - Arbitrary (0) imputation: variables = ['2ndFlrSF', 'MasVnrArea']
    - Categorical imputation (most frequent):
        - fill_value='Unf': variables = ['GarageFinish', 'BsmtFinType1']
        - fill_value='No' : variables = ['BsmtExposure']
    - Dropped variables: features_to_drop = ['EnclosedPorch', 'WoodDeckSF']
- Split dataset into training and test sets
- Built a cleaning pipeline for future reuse
- Cleaned data saved to `outputs/datasets/cleaned/`
- Data cleaning pipeline saved to `outputs/ml_pipeline/data_cleaning/`

**Next Steps**:

Move to Data Study (EDA) Notebook to analyze feature–target relationships and generate visual insights for the dashboard.